In [1]:
import os
import json
from glob import glob

In [2]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


def sqlite_main():
    database = r"person.db"

    sql_create_images_table = """ CREATE TABLE IF NOT EXISTS images (
                                        id integer PRIMARY KEY,
                                        dataset_name text NOT NULL,
                                        image_name text NOT NULL,
                                        rel_flag BOOLEAN NOT NULL CHECK (rel_flag IN (0,1))
                                    ); """

    sql_create_objects_table = """CREATE TABLE IF NOT EXISTS objects (
                                    id integer PRIMARY KEY,
                                    object_name text NOT NULL,
                                    score real NOT NULL,
                                    dataset_name text NOT NULL,
                                    image_name text NOT NULL,
                                    image_id integer NOT NULL,
                                    FOREIGN KEY (image_id) REFERENCES images (id)
                                );"""
    

    sql_create_relations_table = """CREATE TABLE IF NOT EXISTS relations (
                                    id integer PRIMARY KEY,
                                    object_1 text NOT NULL,
                                    relation text NOT NULL,
                                    object_2 text NOT NULL,
                                    relation_score real NOT NULL,
                                    object1_score real NOT NULL,
                                    object2_score real NOT NULL,
                                    dataset_name text NOT NULL,
                                    image_name text NOT NULL,
                                    image_id integer NOT NULL,
                                    FOREIGN KEY (image_id) REFERENCES images (id)
                                );"""


    # create a database connection
    conn = create_connection(database)
    
    # create tables
    if conn is not None:
        # create images table
        create_table(conn, sql_create_images_table)
    
        # create objects table
        create_table(conn, sql_create_objects_table)
    
        # create relations table
        create_table(conn, sql_create_relations_table)
    else:
        print("Error! cannot create the database connection.")

sqlite_main()

In [3]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn


def insert_image(conn, image):
    """
    Create a new project into the projects table
    :param conn:
    :param image:
    :return: image id
    """
    sql = ''' INSERT INTO images(dataset_name, image_name,rel_flag)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, image)
    conn.commit()
    return cur.lastrowid


def insert_object(conn, _object):
    """
    Create a new object
    :param conn:
    :param object:
    :return:
    """

    sql = ''' INSERT INTO objects(object_name,score,dataset_name,image_name,image_id)
              VALUES(?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, _object)
    conn.commit()
    return cur.lastrowid


def insert_relation(conn, _rel):
    """
    Create a new relation
    :param conn:
    :param _rel:
    :return:
    """

    sql = ''' INSERT INTO relations(object_1,relation,object_2,relation_score,object1_score,object2_score,dataset_name,image_name,image_id)
              VALUES(?,?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, _rel)
    conn.commit()
    return cur.lastrowid



In [4]:
def insert_data(data_dir, e_check, dataset='Eurocity'):
    database = r"person.db"
    # create a database connection
    conn = create_connection(database)
    
    box_topk = 30  # select top k bounding boxes
    rel_topk = 20  # select top k relationships

    for d in data_dir:

        print(f"we are trying to filter prediction for {d}")

        info_path = os.path.join(e_check, d, 'custom_data_info.json')
        prediction_file = os.path.join(e_check, d, 'custom_prediction.json')
        custom_prediction = json.load(open(prediction_file))
        custom_data_info = json.load(open(info_path))

        ind_to_classes = custom_data_info['ind_to_classes']
        ind_to_predicates = custom_data_info['ind_to_predicates']

        human_labels = ['woman', 'men', 'boy', 'kid', 'child', 'guy', 'man', 'person', 'girl', 'lady', 'people']

        for image_idx in range(len(custom_data_info['idx_to_files'])):
            file_name, _ = os.path.basename(custom_data_info['idx_to_files'][image_idx]).rsplit(".")
            image_path = custom_data_info['idx_to_files'][image_idx]
            boxes = custom_prediction[str(image_idx)]['bbox'][:box_topk]
            box_labels = custom_prediction[str(image_idx)]['bbox_labels'][:box_topk]
            box_scores = custom_prediction[str(image_idx)]['bbox_scores'][:box_topk]
            all_rel_labels = custom_prediction[str(image_idx)]['rel_labels']
            all_rel_scores = custom_prediction[str(image_idx)]['rel_scores']
            all_rel_pairs = custom_prediction[str(image_idx)]['rel_pairs']

            for i in range(len(box_labels)):
                box_labels[i] = ind_to_classes[box_labels[i]]

            rel_labels_list = []
            rel_scores = []

            new_rel_pairs = []
            new_rel_labels = []
            new_rel_scores = []

            for i in range(len(all_rel_pairs)):
                if all_rel_pairs[i][0] < box_topk and all_rel_pairs[i][1] < box_topk:
                    if box_labels[all_rel_pairs[i][0]] in human_labels or box_labels[all_rel_pairs[i][1]] in human_labels:
#                       label = str(all_rel_pairs[i][0]) + '_' + box_labels[all_rel_pairs[i][0]] + ' => ' + ind_to_predicates[all_rel_labels[i]] + ' => ' + str(all_rel_pairs[i][1]) + '_' + box_labels[all_rel_pairs[i][1]]
                        new_rel_pairs.append(all_rel_pairs[i])
                        new_rel_labels.append(all_rel_labels[i])
                        new_rel_scores.append(all_rel_scores[i])

            h_flag = False
            new_rel_scores = new_rel_scores[:rel_topk]
            new_rel_pairs = new_rel_pairs[:rel_topk]
            new_rel_labels = new_rel_labels[:rel_topk]
            for i in range(len(new_rel_pairs)):
                if new_rel_pairs[i][0] < box_topk and new_rel_pairs[i][1] < box_topk:
                    if box_labels[new_rel_pairs[i][0]] in human_labels or box_labels[new_rel_pairs[i][1]] in human_labels:
                        h_flag = True
                    r_list = [box_labels[new_rel_pairs[i][0]], ind_to_predicates[new_rel_labels[i]],
                              box_labels[new_rel_pairs[i][1]], box_scores[new_rel_pairs[i][0]],
                              box_scores[new_rel_pairs[i][1]]
                             ]
                    rel_labels_list.append(r_list)

#             print(rel_labels_list)
#             print(new_rel_scores)
#             print(box_labels)
#             print(box_scores)
            if h_flag:
                rel_flag = '1'
            else:
                rel_flag = '0'

            image_ = (dataset, file_name, rel_flag)
#             print(image_)
#             image_id = 0
            image_id = insert_image(conn, image_)

            for idx, _obj in enumerate(box_labels):
                obj_tuples = (_obj, box_scores[idx], dataset, file_name, image_id)
#                 print(obj_tuples)
                insert_object(conn, obj_tuples)

            # relations
            for idx, rel in enumerate(rel_labels_list):
                rel_ = (rel[0], rel[1], rel[2], new_rel_scores[idx], rel[3], rel[4], dataset, file_name, image_id)
#                 print(rel_)
                insert_relation(conn, rel_)
#             if h_flag:
#                 break
#         break

In [5]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint/coco"

e_image_dir = glob(f"{e_check}/*")
train_dir_name = [x.rsplit("/", 1)[1] for x in e_image_dir]
print(train_dir_name)

insert_data(train_dir_name, e_check, dataset='COCO')

['train_2', 'train_1', 'train_3']
we are trying to filter prediction for train_2
we are trying to filter prediction for train_1
we are trying to filter prediction for train_3


In [9]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint/eurocity"

e_image_dir = glob(f"{e_check}/*")
train_dir_name = [x.rsplit("/", 1)[1] for x in e_image_dir]
# print(train_dir_name)

insert_data(train_dir_name, e_check, dataset='Eurocity')


we are trying to filter prediction for ljubljana_day
we are trying to filter prediction for potsdam_day
we are trying to filter prediction for marseille_day
we are trying to filter prediction for hamburg_day
we are trying to filter prediction for firenze_day
we are trying to filter prediction for nuernberg_day
we are trying to filter prediction for dresden_day
we are trying to filter prediction for szczecin_day
we are trying to filter prediction for amsterdam_day
we are trying to filter prediction for brno_day
we are trying to filter prediction for barcelona_day
we are trying to filter prediction for roma_day
we are trying to filter prediction for prague_day
we are trying to filter prediction for budapest_day
we are trying to filter prediction for stuttgart_day
we are trying to filter prediction for pisa_day
we are trying to filter prediction for wuerzburg_day
we are trying to filter prediction for zagreb_day
we are trying to filter prediction for torino_day
we are trying to filter pre

In [6]:
# check_data
e_check = "/home/test/Scene-Graph/Scene/checkpoint/eurocity"

e_image_dir = glob(f"{e_check}/*")
train_dir_name = [x.rsplit("/", 1)[1] for x in e_image_dir]
# print(train_dir_name)

check_data(train_dir_name, e_check, dataset='Eurocity')


we are trying to filter prediction for ljubljana_day
('Eurocity', 'ljubljana_00599', '0')
('car', 0.8391796946525574, 'Eurocity', 'ljubljana_00599', 0)
('car', 0.8129258751869202, 'Eurocity', 'ljubljana_00599', 0)
('building', 0.6957226395606995, 'Eurocity', 'ljubljana_00599', 0)
('car', 0.6710112690925598, 'Eurocity', 'ljubljana_00599', 0)
('car', 0.6644753217697144, 'Eurocity', 'ljubljana_00599', 0)
('building', 0.6599908471107483, 'Eurocity', 'ljubljana_00599', 0)
('street', 0.572593629360199, 'Eurocity', 'ljubljana_00599', 0)
('car', 0.5186793804168701, 'Eurocity', 'ljubljana_00599', 0)
('building', 0.5153779983520508, 'Eurocity', 'ljubljana_00599', 0)
('window', 0.4156506359577179, 'Eurocity', 'ljubljana_00599', 0)
('window', 0.3839051127433777, 'Eurocity', 'ljubljana_00599', 0)
('window', 0.35268473625183105, 'Eurocity', 'ljubljana_00599', 0)
('window', 0.32644158601760864, 'Eurocity', 'ljubljana_00599', 0)
('pole', 0.2662825882434845, 'Eurocity', 'ljubljana_00599', 0)
('tire', 0

('Eurocity', 'ljubljana_00514', '1')
('building', 0.6286448836326599, 'Eurocity', 'ljubljana_00514', 34)
('car', 0.6134521961212158, 'Eurocity', 'ljubljana_00514', 34)
('sign', 0.5615260004997253, 'Eurocity', 'ljubljana_00514', 34)
('man', 0.5554975271224976, 'Eurocity', 'ljubljana_00514', 34)
('street', 0.49516990780830383, 'Eurocity', 'ljubljana_00514', 34)
('car', 0.48464247584342957, 'Eurocity', 'ljubljana_00514', 34)
('window', 0.4340071678161621, 'Eurocity', 'ljubljana_00514', 34)
('tree', 0.3995434045791626, 'Eurocity', 'ljubljana_00514', 34)
('house', 0.3837282061576843, 'Eurocity', 'ljubljana_00514', 34)
('bike', 0.33757922053337097, 'Eurocity', 'ljubljana_00514', 34)
('tree', 0.3029665946960449, 'Eurocity', 'ljubljana_00514', 34)
('roof', 0.2575432062149048, 'Eurocity', 'ljubljana_00514', 34)
('fence', 0.2385365515947342, 'Eurocity', 'ljubljana_00514', 34)
('car', 0.2148653268814087, 'Eurocity', 'ljubljana_00514', 34)
('building', 0.199143186211586, 'Eurocity', 'ljubljana_005

('Eurocity', 'ljubljana_00159', '0')
('sign', 0.7509090304374695, 'Eurocity', 'ljubljana_00159', 66)
('car', 0.7401553988456726, 'Eurocity', 'ljubljana_00159', 66)
('street', 0.5833175778388977, 'Eurocity', 'ljubljana_00159', 66)
('car', 0.4802614748477936, 'Eurocity', 'ljubljana_00159', 66)
('tree', 0.4320579469203949, 'Eurocity', 'ljubljana_00159', 66)
('car', 0.41147059202194214, 'Eurocity', 'ljubljana_00159', 66)
('tree', 0.38957351446151733, 'Eurocity', 'ljubljana_00159', 66)
('tree', 0.38548600673675537, 'Eurocity', 'ljubljana_00159', 66)
('sign', 0.30986008048057556, 'Eurocity', 'ljubljana_00159', 66)
('pole', 0.27003735303878784, 'Eurocity', 'ljubljana_00159', 66)
('pole', 0.2566392719745636, 'Eurocity', 'ljubljana_00159', 66)
('vehicle', 0.23145855963230133, 'Eurocity', 'ljubljana_00159', 66)
('pole', 0.22732973098754883, 'Eurocity', 'ljubljana_00159', 66)
('sidewalk', 0.21720272302627563, 'Eurocity', 'ljubljana_00159', 66)
('tree', 0.209517702460289, 'Eurocity', 'ljubljana_00

('Eurocity', 'ljubljana_00565', '0')
('sign', 0.45058831572532654, 'Eurocity', 'ljubljana_00565', 96)
('post', 0.42806604504585266, 'Eurocity', 'ljubljana_00565', 96)
('sign', 0.35707712173461914, 'Eurocity', 'ljubljana_00565', 96)
('tree', 0.3314819037914276, 'Eurocity', 'ljubljana_00565', 96)
('tree', 0.3224128186702728, 'Eurocity', 'ljubljana_00565', 96)
('tree', 0.31795787811279297, 'Eurocity', 'ljubljana_00565', 96)
('street', 0.3176111876964569, 'Eurocity', 'ljubljana_00565', 96)
('tree', 0.3034394383430481, 'Eurocity', 'ljubljana_00565', 96)
('light', 0.2752038538455963, 'Eurocity', 'ljubljana_00565', 96)
('tree', 0.21544702351093292, 'Eurocity', 'ljubljana_00565', 96)
('trunk', 0.20980097353458405, 'Eurocity', 'ljubljana_00565', 96)
('pole', 0.190341517329216, 'Eurocity', 'ljubljana_00565', 96)
('tree', 0.1898241639137268, 'Eurocity', 'ljubljana_00565', 96)
('tree', 0.1874195635318756, 'Eurocity', 'ljubljana_00565', 96)
('sidewalk', 0.16551290452480316, 'Eurocity', 'ljubljana_0

('Eurocity', 'ljubljana_00594', '0')
('sign', 0.7748494744300842, 'Eurocity', 'ljubljana_00594', 126)
('street', 0.6731175184249878, 'Eurocity', 'ljubljana_00594', 126)
('building', 0.5374028086662292, 'Eurocity', 'ljubljana_00594', 126)
('tree', 0.5157580971717834, 'Eurocity', 'ljubljana_00594', 126)
('pole', 0.3153650164604187, 'Eurocity', 'ljubljana_00594', 126)
('sign', 0.27530696988105774, 'Eurocity', 'ljubljana_00594', 126)
('tree', 0.25210049748420715, 'Eurocity', 'ljubljana_00594', 126)
('pole', 0.2215162068605423, 'Eurocity', 'ljubljana_00594', 126)
('car', 0.21238797903060913, 'Eurocity', 'ljubljana_00594', 126)
('sign', 0.2052403688430786, 'Eurocity', 'ljubljana_00594', 126)
('tree', 0.19285960495471954, 'Eurocity', 'ljubljana_00594', 126)
('tree', 0.1924360990524292, 'Eurocity', 'ljubljana_00594', 126)
('tree', 0.16648122668266296, 'Eurocity', 'ljubljana_00594', 126)
('pole', 0.12933163344860077, 'Eurocity', 'ljubljana_00594', 126)
('light', 0.11877389997243881, 'Eurocity',

...

('Eurocity', 'ljubljana_00197', '0')
('car', 0.7816965579986572, 'Eurocity', 'ljubljana_00197', 778)
('car', 0.7574694156646729, 'Eurocity', 'ljubljana_00197', 778)
('car', 0.7139674425125122, 'Eurocity', 'ljubljana_00197', 778)
('car', 0.7013368606567383, 'Eurocity', 'ljubljana_00197', 778)
('street', 0.6491742730140686, 'Eurocity', 'ljubljana_00197', 778)
('building', 0.607453465461731, 'Eurocity', 'ljubljana_00197', 778)
('flag', 0.5779100060462952, 'Eurocity', 'ljubljana_00197', 778)
('building', 0.5644441246986389, 'Eurocity', 'ljubljana_00197', 778)
('car', 0.4822824001312256, 'Eurocity', 'ljubljana_00197', 778)
('sign', 0.4240260720252991, 'Eurocity', 'ljubljana_00197', 778)
('building', 0.35427162051200867, 'Eurocity', 'ljubljana_00197', 778)
('tire', 0.3103691637516022, 'Eurocity', 'ljubljana_00197', 778)
('tire', 0.27572062611579895, 'Eurocity', 'ljubljana_00197', 778)
('sign', 0.24968068301677704, 'Eurocity', 'ljubljana_00197', 778)
('building', 0.2482539415359497, 'Eurocity

('person', 'walking on', 'sidewalk', 0.7908967733383179, 0.11536243557929993, 0.2600955069065094, 'Eurocity', 'ljubljana_00030', 808)
('person', 'walking on', 'sidewalk', 0.7904060482978821, 0.1201792061328888, 0.2600955069065094, 'Eurocity', 'ljubljana_00030', 808)
('person', 'walking on', 'sidewalk', 0.7894538044929504, 0.1201792061328888, 0.172021746635437, 'Eurocity', 'ljubljana_00030', 808)
('person', 'walking on', 'sidewalk', 0.7887110114097595, 0.16113951802253723, 0.2600955069065094, 'Eurocity', 'ljubljana_00030', 808)
('people', 'walking on', 'sidewalk', 0.7854951024055481, 0.16869120299816132, 0.172021746635437, 'Eurocity', 'ljubljana_00030', 808)
('people', 'walking on', 'sidewalk', 0.7823902368545532, 0.16869120299816132, 0.2600955069065094, 'Eurocity', 'ljubljana_00030', 808)
('people', 'walking on', 'sidewalk', 0.780592679977417, 0.35648566484451294, 0.172021746635437, 'Eurocity', 'ljubljana_00030', 808)
('people', 'walking on', 'sidewalk', 0.7787811160087585, 0.356485664

('Eurocity', 'ljubljana_00533', '0')
('car', 0.8200916647911072, 'Eurocity', 'ljubljana_00533', 840)
('car', 0.8024158477783203, 'Eurocity', 'ljubljana_00533', 840)
('building', 0.6969786882400513, 'Eurocity', 'ljubljana_00533', 840)
('street', 0.6824123859405518, 'Eurocity', 'ljubljana_00533', 840)
('sign', 0.6229856014251709, 'Eurocity', 'ljubljana_00533', 840)
('building', 0.6117563247680664, 'Eurocity', 'ljubljana_00533', 840)
('sign', 0.43030521273612976, 'Eurocity', 'ljubljana_00533', 840)
('pole', 0.3475710451602936, 'Eurocity', 'ljubljana_00533', 840)
('window', 0.3393864333629608, 'Eurocity', 'ljubljana_00533', 840)
('building', 0.33638301491737366, 'Eurocity', 'ljubljana_00533', 840)
('pole', 0.2921469509601593, 'Eurocity', 'ljubljana_00533', 840)
('window', 0.2701737880706787, 'Eurocity', 'ljubljana_00533', 840)
('sign', 0.22774188220500946, 'Eurocity', 'ljubljana_00533', 840)
('building', 0.20949536561965942, 'Eurocity', 'ljubljana_00533', 840)
('sign', 0.20318391919136047,

('Eurocity', 'ljubljana_00541', '0')
('tree', 0.5819445848464966, 'Eurocity', 'ljubljana_00541', 870)
('tree', 0.5412914156913757, 'Eurocity', 'ljubljana_00541', 870)
('tree', 0.23162926733493805, 'Eurocity', 'ljubljana_00541', 870)
('street', 0.2079050987958908, 'Eurocity', 'ljubljana_00541', 870)
('fence', 0.1718006432056427, 'Eurocity', 'ljubljana_00541', 870)
('tree', 0.1703118085861206, 'Eurocity', 'ljubljana_00541', 870)
('fence', 0.1353355050086975, 'Eurocity', 'ljubljana_00541', 870)
('tree', 0.13287848234176636, 'Eurocity', 'ljubljana_00541', 870)
('hill', 0.12615393102169037, 'Eurocity', 'ljubljana_00541', 870)
('tree', 0.11919301003217697, 'Eurocity', 'ljubljana_00541', 870)
('tree', 0.10761905461549759, 'Eurocity', 'ljubljana_00541', 870)
('tree', 0.09860727936029434, 'Eurocity', 'ljubljana_00541', 870)
('tree', 0.09372459352016449, 'Eurocity', 'ljubljana_00541', 870)
('branch', 0.09165108948945999, 'Eurocity', 'ljubljana_00541', 870)
('hill', 0.08556199818849564, 'Eurocity

KeyboardInterrupt: 

In [ ]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint/caltech_output"

e_image_dir = glob(f"{e_check}/*")
train_dir_name = [x.rsplit("/", 1)[1] for x in e_image_dir]
print(train_dir_name)

insert_data(train_dir_name, e_check, dataset='Caltech')


In [13]:
! ls "/home/test/Scene-Graph/Scene/checkpoint/custom_output_path"

custom_data_info.json  custom_prediction.json


In [14]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint"
train_dir_name = ["custom_output_path"]
insert_data(train_dir_name, e_check, dataset='CityPerson')

we are trying to filter prediction for custom_output_path


In [15]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint/CrowdHuman"

e_image_dir = glob(f"{e_check}/*")
train_dir_name = [x.rsplit("/", 1)[1] for x in e_image_dir]
print(train_dir_name)

insert_data(train_dir_name, e_check, dataset='CrowdHuman')

['train_2', 'train_5', 'train_4', 'train_1', 'train_3']
we are trying to filter prediction for train_2
we are trying to filter prediction for train_5
we are trying to filter prediction for train_4
we are trying to filter prediction for train_1
we are trying to filter prediction for train_3


In [16]:
e_check = "/home/test/Scene-Graph/Scene/checkpoint/WiderPerson"

e_image_dir = glob(f"{e_check}/*")
train_dir_name = [x.rsplit("/", 1)[1] for x in e_image_dir]
print(train_dir_name)

insert_data(train_dir_name, e_check, dataset='WiderPerson')

['train_2', 'train_1', 'train_3']
we are trying to filter prediction for train_2
we are trying to filter prediction for train_1
we are trying to filter prediction for train_3
